In [2]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
import glob
from datetime import timedelta

In [39]:
df = pd.read_csv('C:/Users/ale/Documents/Tec/PD sensor extracted data/main_sleep/18/sleepPD.csv')
df['t'] = pd.to_datetime(df['t'])
df.head()

,t,x,y,z,classification,arm_angle,change
0,2021-03-18 23:00:00.039,0.768,0.096,-0.576,Wake,-36.656933,0.000000
1,2021-03-18 23:00:00.069,0.768,0.096,-0.576,Wake,-36.656933,0.000000
2,2021-03-18 23:00:00.109,0.768,0.096,-0.576,Wake,-36.656933,0.000000
3,2021-03-18 23:00:00.139,0.768,0.096,-0.576,Wake,-36.656933,0.000000
4,2021-03-18 23:00:00.179,0.768,0.096,-0.608,Wake,-38.151524,-1.494591


In [40]:
df1 = df[df.classification != 'Wake']
df1

,t,x,y,z,classification,arm_angle,change
25837,2021-03-18 23:14:50.167,0.320,0.672,-0.672,Sleep,-42.077634,-1.386478
25838,2021-03-18 23:14:50.207,0.320,0.672,-0.672,Sleep,-42.077634,0.000000
25839,2021-03-18 23:14:50.237,0.320,0.672,-0.672,Sleep,-42.077634,0.000000
25840,2021-03-18 23:14:50.277,0.320,0.672,-0.640,Sleep,-40.691156,1.386478
25841,2021-03-18 23:14:50.307,0.320,0.640,-0.640,Sleep,-41.810315,-1.119159
...,...,...,...,...,...,...,...
103170,2021-03-18 23:59:45.996,-0.224,0.960,0.352,Sleep,19.650381,0.000000
103171,2021-03-18 23:59:46.026,-0.224,0.960,0.352,Sleep,19.650381,0.000000
103172,2021-03-18 23:59:46.066,-0.224,0.960,0.352,Sleep,19.650381,0.000000
103173,2021-03-18 23:59:46.096,-0.224,0.960,0.384,Sleep,21.282729,1.632347


In [41]:
def stats(df):
    mean_arm_angle = df['arm_angle'].mean()
    std_arm_angle = df['arm_angle'].std() 
    mean_change = df['change'].mean() 
    std_change = df['change'].std()
    episode_len = df['t'].iloc[-1] - df['t'].iloc[0]
    num_data_points = len(df)
    return mean_arm_angle, std_arm_angle, mean_change, std_change, episode_len, num_data_points

def sleep_interuptions(df, episode_len):
    len_interuptions = []
    counter = 0 
    for i in range(len(df)-1):
        if (df['t'].iloc[i+1] - df['t'].iloc[i]) > timedelta(minutes=5):
            counter += 1
            len_interuptions.append(df['t'].iloc[i+1] - df['t'].iloc[i])
        else:
            pass
    time_wake = timedelta(milliseconds=0)
    for i in range(len(len_interuptions)):
        time_wake += len_interuptions[i]
    time_sleep = episode_len - time_wake
    return counter, len_interuptions, time_sleep, time_wake

In [42]:
mean_arm_angle, std_arm_angle, mean_change, std_change, episode_len, num_data_points = stats(df1)
counter, len_interuptions, time_sleep, time_wake = sleep_interuptions(df1, episode_len)

In [43]:
d1 = {'Total Time': [episode_len], 'Data Points': [num_data_points], 'Num Sleep Interuptions': [counter], 
     'Time Asleep': [time_sleep], 'Time Awake': [time_wake], 'Mean Arm Angle': [mean_arm_angle], 
     'STD Arm Angle': [std_arm_angle], 'Mean Change in Arm Angle': [mean_change], 'STD Change in Arm Angle': [std_change]}
df2 = pd.DataFrame(data=d1)
df2

,Total Time,Data Points,Num Sleep Interuptions,Time Asleep,Time Awake,Mean Arm Angle,STD Arm Angle,Mean Change in Arm Angle,STD Change in Arm Angle
0,0 days 00:44:55.969000,61854,1,0 days 00:35:55.694000,0 days 00:09:00.275000,-11.401044,29.455561,0.000393,5.668311


In [44]:
df2.to_csv('C:/Users/ale/Documents/Tec/PD sensor extracted data/main_sleep/18/sleepPD_stats.csv', index = False)